# Usage

## Using the included categorizations

In the `climate_categories` package, the categorizations are available
directly at the top-level namespace, and as a dictionary in `.cats`:

In [ ]:
import climate_categories

climate_categories.IPCC2006.title

In [ ]:
climate_categories.cats["IPCC2006"].title

Metadata for each categorization are accessible as properties:

In [ ]:
print(climate_categories.IPCC2006.name)
print(climate_categories.IPCC2006.title)
print(climate_categories.IPCC2006.comment)
print(climate_categories.IPCC2006.references)
print(climate_categories.IPCC2006.institution)
print(climate_categories.IPCC2006.last_update)
print(climate_categories.IPCC2006.version)

The categorization can be used as a dictionary to query the meaning
of category codes:

In [ ]:
climate_categories.IPCC2006["1A"]

For hierarchical categorizations, you can also query for parent and child
categories:

In [ ]:
climate_categories.IPCC2006.children("1A")

In [ ]:
climate_categories.IPCC2006.parents("1A")

Finally, you can check if a categorization is hierarchical, and
for hierarchical categorizations you can check if the sum of all
child categories should be equal to the sum of parent categories:

In [ ]:
print(f"Hierachical: {climate_categories.IPCC2006.hierarchical}")
print(f"Total sum: {climate_categories.IPCC2006.total_sum}")

## Extending categorizations

Often, you want to use a common categorization, but for one reason or
another, you have to add a couple of categories. This is possible:

In [ ]:
IPCC2006_PRIMAP = climate_categories.IPCC2006.extend(
    name="PRIMAP",
    categories={
        "MLULUCF": "Total emissions from land use, land use change, and forestry"
    },
)

print(IPCC2006_PRIMAP.name)
print(IPCC2006_PRIMAP.title)
print(IPCC2006_PRIMAP.comment)

In this example, only a top-level category was added - if you need to
add relationships, also supply a dictionary giving the children of each
category:

In [ ]:
IPCC2006_PRIMAP = climate_categories.IPCC2006.extend(
    name="PRIMAP",
    categories={
        "M0": "Total",
        "MLULUCF": "Total emissions from land use, land use change, and forestry",
        "MXLULUCF": "Total emissions excluding those from land use, land use change, and forestry",
    },
    children={"M0": ["MLULUCF", "MXLULUCF"]},
)

## Pandas integration

For each categorization, the categories are also available as a pandas
DataFrame (you need to have pandas installed):

In [ ]:
climate_categories.IPCC2006.df